# Tutorial: Async Data Processing with alcall/bcall

**Category**: ln Utilities
**Difficulty**: Intermediate
**Time**: 20-25 minutes

## Problem Statement

Processing large datasets with I/O-bound operations (API calls, LLM inference, database queries) sequentially wastes time. Naive parallelization without rate limiting overwhelms services.

**Why This Matters**:
- **Performance**: Concurrent processing reduces wall time by 10-100×
- **Resource Management**: Rate limiting prevents service disruption
- **Reliability**: Retry with backoff handles transient failures

**What You'll Build**:
Production-ready async data processing pipelines using `alcall()` (async map) and `bcall()` (batch processor).

## Prerequisites

**Prior Knowledge**:
- Python async/await fundamentals
- Basic understanding of concurrency concepts

**Required Packages**:
```bash
pip install lionherd-core  # >=0.1.0
```

In [1]:
# Standard library
import asyncio
from dataclasses import dataclass
from time import time

# lionherd-core
from lionherd_core.ln import alcall, bcall

## Solution Overview

We'll implement async data processing patterns:

1. **alcall Basics**: Concurrent mapping with concurrency control
2. **Rate Limiting**: Control concurrent requests to protect services
3. **bcall Batching**: Process large datasets in memory-efficient batches
4. **Production Patterns**: Retry strategies, error handling, fallbacks

**Key Components**:
- `alcall`: Async map with retry, timeout, and concurrency control
- `bcall`: Batch processor (async generator yielding results)

**Pattern**: Process data concurrently while respecting service limits and handling failures gracefully.

### Step 1: alcall Basics

Understand concurrent async mapping with timeout and concurrency control.

**Key Point**: `alcall()` processes items concurrently while preserving input order.

In [2]:
# Simulated async API call
async def fetch_data(item_id: int) -> dict:
    """Simulate API call with variable latency."""
    await asyncio.sleep(0.1)  # Simulate network delay
    return {"id": item_id, "data": f"result_{item_id}"}


async def demo_basic():
    # Sequential: ~1 second for 10 items
    items = list(range(10))

    start = time()
    # Concurrent processing (no limit)
    results = await alcall(items, fetch_data)
    elapsed = time() - start

    print(f"Processed {len(results)} items in {elapsed:.2f}s")
    print(f"Results: {results[:3]}...")  # First 3 results
    print(f"Speedup: ~{10 * 0.1 / elapsed:.1f}x vs sequential")


await demo_basic()

Processed 10 items in 0.10s
Results: [{'id': 0, 'data': 'result_0'}, {'id': 1, 'data': 'result_1'}, {'id': 2, 'data': 'result_2'}]...
Speedup: ~9.6x vs sequential


### Step 2: Concurrency Control with max_concurrent

Limit concurrent operations to prevent overwhelming services.

**Key Point**: `max_concurrent` uses semaphore to cap active tasks (critical for rate limiting).

In [3]:
async def simulate_api_call(item_id: int) -> dict:
    """Simulate API with rate limits (max 5 concurrent)."""
    await asyncio.sleep(0.2)
    return {"id": item_id, "status": "success"}


async def demo_rate_limiting():
    items = list(range(20))

    # Without limit: might overwhelm API
    start = time()
    results_unlimited = await alcall(items, simulate_api_call)
    elapsed_unlimited = time() - start

    # With limit: max 5 concurrent (respects API limits)
    start = time()
    results_limited = await alcall(
        items,
        simulate_api_call,
        max_concurrent=5,  # Semaphore: only 5 tasks active at once
    )
    elapsed_limited = time() - start

    print(f"Unlimited: {elapsed_unlimited:.2f}s (all 20 concurrent)")
    print(f"Limited (5): {elapsed_limited:.2f}s (batches of 5)")
    print(f"Success: unlimited={len(results_unlimited)}, limited={len(results_limited)}")


await demo_rate_limiting()

Unlimited: 0.20s (all 20 concurrent)
Limited (5): 0.80s (batches of 5)
Success: unlimited=20, limited=20


### Step 3: Retry Strategies

Handle transient failures with exponential backoff.

**Key Point**: `retry_attempts` + `retry_backoff` implements resilient error recovery.

In [4]:
# Simulated flaky API
_call_counts = {}


async def flaky_api(item_id: int) -> dict:
    """Fails first 2 attempts, succeeds on 3rd."""
    _call_counts[item_id] = _call_counts.get(item_id, 0) + 1

    if _call_counts[item_id] < 3:
        raise ConnectionError(f"Transient failure for {item_id}")

    return {"id": item_id, "attempts": _call_counts[item_id]}


async def demo_retry():
    _call_counts.clear()  # Reset counts
    items = [1, 2, 3]

    # With retry: succeeds after 3 attempts
    results = await alcall(
        items,
        flaky_api,
        retry_attempts=3,  # Max 3 retries
        retry_initial_delay=0.1,  # Start with 100ms
        retry_backoff=2.0,  # Double delay each retry (100ms → 200ms → 400ms)
        return_exceptions=False,  # Raise if all retries fail
    )

    print(f"Results: {results}")
    print("All succeeded after retries")


await demo_retry()

Results: [{'id': 1, 'attempts': 3}, {'id': 2, 'attempts': 3}, {'id': 3, 'attempts': 3}]
All succeeded after retries


### Step 4: Timeout Protection

Prevent indefinite hangs with per-call timeouts.

**Key Point**: `retry_timeout` bounds execution time per item (uses `move_on_after`).

In [5]:
async def slow_operation(item_id: int) -> dict:
    """Simulates operation that might hang."""
    if item_id == 5:
        await asyncio.sleep(10)  # Simulates hang
    else:
        await asyncio.sleep(0.1)
    return {"id": item_id}


async def demo_timeout():
    items = [1, 2, 5, 7]  # Item 5 will timeout

    # With timeout: prevents hanging on item 5
    results = await alcall(
        items,
        slow_operation,
        retry_timeout=1.0,  # Max 1 second per call
        return_exceptions=True,  # Return TimeoutError instead of raising
    )

    for i, result in zip(items, results, strict=True):
        if isinstance(result, TimeoutError):
            print(f"Item {i}: ✗ Timeout")
        else:
            print(f"Item {i}: ✓ Success")


await demo_timeout()

Item 1: ✓ Success
Item 2: ✓ Success
Item 5: ✗ Timeout
Item 7: ✓ Success


### Step 5: bcall for Batch Processing

Process large datasets in memory-efficient batches.

**Key Point**: `bcall()` is an async generator yielding batch results (prevents loading all results in memory).

In [6]:
async def process_item(item: int) -> dict:
    """Simulate processing."""
    await asyncio.sleep(0.05)
    return {"id": item, "processed": True}


async def demo_batching():
    # Large dataset (1000 items)
    items = list(range(100))

    # Process in batches of 10
    batch_num = 0
    total_processed = 0

    async for batch_results in bcall(
        items,
        process_item,
        batch_size=10,  # 10 items per batch
        max_concurrent=5,  # 5 concurrent within each batch
    ):
        batch_num += 1
        total_processed += len(batch_results)

        # Process batch results immediately (streaming pattern)
        if batch_num == 1:
            print(f"Batch {batch_num}: {len(batch_results)} items")
            print(f"  First result: {batch_results[0]}")

    print(f"\nProcessed {total_processed} items in {batch_num} batches")


await demo_batching()

Batch 1: 10 items
  First result: {'id': 0, 'processed': True}

Processed 100 items in 10 batches


### Step 6: Throttling Requests

Add delay between starting tasks to smooth out request rate.

**Key Point**: `throttle_period` staggers task starts (useful for strict rate limits).

In [7]:
async def rate_limited_api(item_id: int) -> dict:
    """API with strict rate limit: 10 requests/second."""
    await asyncio.sleep(0.05)
    return {"id": item_id}


async def demo_throttling():
    items = list(range(10))

    start = time()
    # Throttle: 0.1s delay between starting each task
    results = await alcall(
        items,
        rate_limited_api,
        throttle_period=0.1,  # 100ms between task starts (10 req/s)
        max_concurrent=3,  # Still limit concurrent
    )
    elapsed = time() - start

    print(f"Processed {len(results)} items in {elapsed:.2f}s")
    print(f"Rate: ~{len(results) / elapsed:.1f} req/s (respects 10 req/s limit)")


await demo_throttling()

Processed 10 items in 0.96s
Rate: ~10.4 req/s (respects 10 req/s limit)


## Complete Working Example

Production-ready LLM batch inference with comprehensive error handling.

In [8]:
"""Production async batch processor."""


@dataclass
class ProcessingConfig:
    max_concurrent: int = 5
    batch_size: int = 10
    retry_attempts: int = 3
    retry_initial_delay: float = 0.5
    retry_backoff: float = 2.0
    timeout_per_item: float = 30.0
    throttle_period: float = 0.1


class BatchProcessor:
    """Production batch processor with retry and rate limiting."""

    def __init__(self, config: ProcessingConfig = None):
        self.config = config or ProcessingConfig()

    async def process_batch(self, items: list, process_func, handle_error=None):
        """Process items in batches with error handling."""
        results = []
        errors = []

        async for batch_results in bcall(
            items,
            process_func,
            batch_size=self.config.batch_size,
            max_concurrent=self.config.max_concurrent,
            retry_attempts=self.config.retry_attempts,
            retry_initial_delay=self.config.retry_initial_delay,
            retry_backoff=self.config.retry_backoff,
            retry_timeout=self.config.timeout_per_item,
            throttle_period=self.config.throttle_period,
            return_exceptions=True,  # Don't fail entire batch on error
        ):
            # Separate successes from failures
            for result in batch_results:
                if isinstance(result, BaseException):
                    errors.append(result)
                    if handle_error:
                        await handle_error(result)
                else:
                    results.append(result)

        return {"results": results, "errors": errors}


# Simulated LLM inference
async def llm_inference(prompt: str) -> dict:
    """Simulate LLM API call."""
    await asyncio.sleep(0.2)  # Simulate inference time
    if "error" in prompt:
        raise ValueError(f"Invalid prompt: {prompt}")
    return {"prompt": prompt, "response": f"Generated from: {prompt}"}


# Usage
async def demo_production():
    prompts = [
        "Explain quantum computing",
        "Write a poem about AI",
        "error: invalid",  # Will fail
        "Summarize this article",
    ] * 5  # 20 prompts total

    config = ProcessingConfig(
        max_concurrent=3,
        batch_size=5,
        retry_attempts=2,
        timeout_per_item=10.0,
    )

    processor = BatchProcessor(config)

    async def log_error(error):
        print(f"  Error: {type(error).__name__}: {error}")

    start = time()
    result = await processor.process_batch(prompts, llm_inference, log_error)
    elapsed = time() - start

    print(f"\nCompleted in {elapsed:.2f}s")
    print(f"Success: {len(result['results'])}")
    print(f"Errors: {len(result['errors'])}")
    print(f"Throughput: {len(prompts) / elapsed:.1f} prompts/s")


await demo_production()

  Error: ValueError: Invalid prompt: error: invalid
  Error: ValueError: Invalid prompt: error: invalid
  Error: ValueError: Invalid prompt: error: invalid
  Error: ValueError: Invalid prompt: error: invalid
  Error: ValueError: Invalid prompt: error: invalid

Completed in 9.43s
Success: 15
Errors: 5
Throughput: 2.1 prompts/s


## Production Considerations

### Error Handling Patterns

```python
# Pattern 1: Return exceptions for partial success
results = await alcall(
    items,
    func,
    return_exceptions=True,  # Get partial results
)

successes = [r for r in results if not isinstance(r, BaseException)]
failures = [r for r in results if isinstance(r, BaseException)]

# Pattern 2: Default value on failure
results = await alcall(
    items,
    func,
    retry_attempts=3,
    retry_default=None,  # Return None after retry exhaustion
)
```

### Performance Tuning

**Concurrency Limits by Use Case**:
- Public APIs: 5-10 concurrent (respect rate limits)
- LLM inference: 3-5 concurrent (cost/throughput balance)
- Database queries: 10-50 concurrent (connection pool size)
- Internal services: 50-200 concurrent (based on capacity)

**Batch Size Guidelines**:
- Memory-bound: Smaller batches (10-50 items)
- CPU-bound: Larger batches (100-1000 items)
- Progress tracking: Smaller batches for frequent updates

### Testing Strategies

```python
async def test_retry_exhaustion():
    """Verify retry attempts are exhausted correctly."""
    
    async def always_fail(x):
        raise ValueError("Always fails")
    
    results = await alcall(
        [1, 2],
        always_fail,
        retry_attempts=2,
        retry_default="FAILED",
    )
    
    assert results == ["FAILED", "FAILED"]


async def test_timeout_triggers():
    """Verify timeout protection works."""
    
    async def slow(x):
        await asyncio.sleep(5)
        return x
    
    results = await alcall(
        [1],
        slow,
        retry_timeout=0.1,
        return_exceptions=True,
    )
    
    assert isinstance(results[0], TimeoutError)
```

## Real-World Use Cases

### Use Case 1: Concurrent API Fetching

```python
async def fetch_user_data(user_ids: list[int]) -> list[dict]:
    """Fetch user data from external API."""
    return await alcall(
        user_ids,
        lambda uid: fetch_api(f"/users/{uid}"),
        max_concurrent=10,  # API rate limit
        retry_attempts=3,
        retry_timeout=5.0,
        retry_backoff=2.0,
    )
```

### Use Case 2: Batch LLM Inference

```python
async def batch_llm_inference(prompts: list[str]):
    """Process prompts in batches with rate limiting."""
    results = []
    
    async for batch in bcall(
        prompts,
        llm_api_call,
        batch_size=10,
        max_concurrent=3,  # Cost control
        retry_attempts=2,
        retry_timeout=30.0,
    ):
        results.extend(batch)
        # Save intermediate results
        await save_checkpoint(results)
    
    return results
```

### Use Case 3: Database Bulk Operations

```python
async def bulk_insert(records: list[dict]):
    """Insert records in batches."""
    async for batch_results in bcall(
        records,
        db.insert,
        batch_size=100,  # Database batch size
        max_concurrent=10,  # Connection pool size
        retry_attempts=3,
    ):
        # Commit per batch
        await db.commit()
```

## Summary

**What You Accomplished**:
- ✅ Implemented concurrent async processing with `alcall()`
- ✅ Added rate limiting and concurrency control
- ✅ Built retry strategies with exponential backoff
- ✅ Processed large datasets with memory-efficient batching via `bcall()`
- ✅ Created production-ready error handling patterns

**Key Takeaways**:
1. **Concurrency != Parallelism**: `max_concurrent` controls active tasks (I/O-bound speedup)
2. **Always use timeouts**: Unbounded operations hang production systems
3. **Retry with backoff**: Exponential backoff handles transient failures gracefully
4. **Batch for memory efficiency**: `bcall()` streams results instead of loading all in memory
5. **Order preservation**: `alcall()`/`bcall()` maintain input order despite concurrent execution

**When to Use**:
- ✅ I/O-bound operations (API calls, database queries, file I/O)
- ✅ LLM inference with rate limits
- ✅ Large datasets requiring batch processing
- ❌ CPU-bound tasks (use multiprocessing instead)
- ❌ Operations requiring strict sequential ordering

## Related Resources

- [alcall API](../../../docs/api/ln/async_call.md)
- [bcall API](../../../docs/api/ln/async_call.md)
- [Async Path Creation](./async_path_creation.ipynb) - Another async utility tutorial
- [Multi-Stage Pipeline](./multistage_pipeline.ipynb) - Composable data pipelines with `lcall()`